<a href="https://colab.research.google.com/github/joe3gold/Google-Chrome-Google-Contacts-Sync/blob/master/WhisperGoogleDrive.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

If you're looking at this on GitHub and new to Python Notebooks or Colab, click the Google Colab badge above 👆

#OpenAI Whisper and Google Drive integration for transcribing audio files

📺 Getting started video: https://youtu.be/yVLhG4-7Sj4

###This notebook will transcribe all the audio files in a Google Drive folder

*Note: This requires giving the application permission to connect to your drive. Only you will have access to the contents of your drive, but please read the warnings carefully.*

This notebook application:
1. Connects to your Google Drive when you give it permission.
2. Creates a WhisperAudio folder and two subfolders (ProcessedAudio and TextFiles.)
3. When you run the application it will search for all the audio files (.mp3 and .m4a) in your WhisperAudio folder, transcribe them and then move the file to /ProcessedAudio and save the transcript to /TextFiles.

###**For faster performance set your runtime to "GPU"**
*Click on "Runtime" in the menu and click "Change runtime type". Select "GPU".*


**Note: If you add a new file after running this application you'll need to remount the drive in step 1 to make them searchable**

##1. Install the required code libraries

In [1]:
!pip install git+https://github.com/openai/whisper.git
!sudo apt update && sudo apt install ffmpeg
!pip install librosa

import whisper
import time
import librosa
import soundfile as sf
import re
import os

# model = whisper.load_model("tiny.en")
# model = whisper.load_model("base.en")
model = whisper.load_model("small.en") # load the small model
# model = whisper.load_model("medium.en")
# model = whisper.load_model("large")

  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-6ynn668t
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-6ynn668t
  Resolved https://github.com/openai/whisper.git to commit c0d2f624c09dc18e709e37c2ad90c039a4eb72a2
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for openai-whisper: filename=openai_whisper-20250625-py3-none-any.whl size=803979 sha256=98a19fecf149fc913d5f3fdb666aa8db022096956211d71ef7a4a9bac9a9c106
  Stored in directory: /tmp/pip-ephem-wheel-cache-tqtkt420/wheels/c3/03/25/5e0ba78bc27a3a089f137c9f1d92fdfce16d06996c071a016c
Successfully built openai-whisper
Get:1 https://cli.github.com/packages stable InRelease [3,917 B]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InR

100%|███████████████████████████████████████| 461M/461M [00:14<00:00, 34.0MiB/s]


##2. Allow access to your Google Drive and add new folders

In [2]:
# Connect Google Drive
from google.colab import drive
drive.mount("/content/drive", force_remount=True) # This will prompt for authorization.

# This will create the WhisperAudio files if they don't exist.
folders =  ["WhisperAudio/", "WhisperAudio/ProcessedAudio/", "WhisperAudio/TextFiles/"]
for folder in folders:
  path = "/content/drive/MyDrive/" + folder
  if not os.path.exists(path): # Create the folder if it does not exist
    os.mkdir(path)

Mounted at /content/drive


##3. Upload any audio files you want transcribed in the "WhisperAudio" folder in your Google Drive.

##4. Let the application search for new files and transcribe the audio files and save them to your Google Drive

In [3]:
# Load all the audio file paths in a Google Drive folder
from google.colab import drive
drive.mount("/content/drive", force_remount=True) # This will prompt for authorization.

# Assuming the audio files are in a folder called "WhisperAudio" in the root of the drive
audio_folder = "/content/drive/MyDrive/WhisperAudio/"

# Get a list of all the file paths and names in the folder
import os
audio_files = []
audio_names = []
for file in os.listdir(audio_folder):
  if file.endswith(".m4a") or file.endswith(".mp3"):
    audio_files.append(audio_folder + file)
    audio_names.append(file)

for f in audio_files:
  print(f)

if len(audio_files) == 0:
  print("You have no files.")

# Loop through the audio files, split each audio file based on pauses in speech then transcribe them with Whisper.
for i, file in enumerate(audio_files): # For each audio file
  print(f"Processing {audio_names[i]}...")
  # Load the audio file and convert it to 16 kHz mono
  audio, sr = librosa.load(file, sr=16000, mono=True)
  # Detect pauses and split the audio. We use a threshold of -30 dB and a minimum pause length of 0.5 seconds.
  pauses = librosa.effects.split(audio, top_db=30, frame_length=2048, hop_length=128)
  # Transcribe each segment and concatenate the results
  transcription = ""
  for start, end in pauses: # For each segment
    segment = audio[start:end]
    # Save the segment as a temporary wav file
    temp_file = "temp.wav"
    sf.write(temp_file, segment, sr, subtype='PCM_16')
    if os.path.getsize(temp_file) > 10000:
      #continue
      # Transcribe the segment with Whisper
      result = model.transcribe(temp_file)
      text = result["text"]
      # Append the text to the transcription
      print(len(transcription.split(" ")), "words processed")
      transcription += text.strip() + " "
      # Delete the temporary file
      os.remove(temp_file)
  # Print the transcription
  print(f"Transcription of {audio_names[i]}:\n")
  print(transcription)
  print("\n")

  # Convert the spaces between sections into paragraph breaks and save the transcription as a txt document in the same folder as MyAudio.
  transcription = re.sub(r"\s\s+", "\n\n", transcription) # Replace multiple spaces with newlines
  text_file = audio_folder + "/TextFiles/" + audio_names[i][:-4] + ".txt" # Create the text file name
  with open(text_file, "w") as f: # Write the transcription to the text file
    f.write(transcription)
  print(f"Saved transcription as {text_file}")

# Move the audio files to "/content/drive/MyDrive/WhisperAudio/Processed"
import shutil
processed_folder = "/content/drive/MyDrive/WhisperAudio/ProcessedAudio/"
if not os.path.exists(processed_folder): # Create the folder if it does not exist
  os.mkdir(processed_folder)
for file in audio_files: # Move each audio file to the processed folder
  shutil.move(file, processed_folder + os.path.basename(file))
  print(f"Moved {file} to {processed_folder}")

Mounted at /content/drive
/content/drive/MyDrive/WhisperAudio/New Recording 3.m4a
/content/drive/MyDrive/WhisperAudio/New Recording or Nezyyyzh.m4a
/content/drive/MyDrive/WhisperAudio/New Ltd Ybhfhflgyly Hubb.m4a
/content/drive/MyDrive/WhisperAudio/New Recording 2.m4a
/content/drive/MyDrive/WhisperAudio/New Recording 8.m4a
/content/drive/MyDrive/WhisperAudio/New Recording 4.m4a
/content/drive/MyDrive/WhisperAudio/New Recording 5.m4a
/content/drive/MyDrive/WhisperAudio/New Recording 6.m4a
/content/drive/MyDrive/WhisperAudio/New Recording 7.m4a
/content/drive/MyDrive/WhisperAudio/New Recording 11.m4a
/content/drive/MyDrive/WhisperAudio/New Recording 10.m4a
/content/drive/MyDrive/WhisperAudio/New Recording 12.m4a
/content/drive/MyDrive/WhisperAudio/New Recording 15.m4a
/content/drive/MyDrive/WhisperAudio/New Recording 14.m4a
/content/drive/MyDrive/WhisperAudio/New Recording 9.m4a
/content/drive/MyDrive/WhisperAudio/New Recording 16.m4a
/content/drive/MyDrive/WhisperAudio/New Recording 13.

/tmp/ipython-input-2017401303.py:27: UserWarning: PySoundFile failed. Trying audioread instead.
  audio, sr = librosa.load(file, sr=16000, mono=True)
/usr/local/lib/python3.12/dist-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)


1 words processed
7 words processed
16 words processed
24 words processed
26 words processed
37 words processed
38 words processed
46 words processed
Transcription of New Recording 3.m4a:

I'm okay, I'm okay, I'm okay. Before we kill you all for not release the We're not gonna hide there, you guys don't... to the Go away and do the Baumashka, do the Shuhan, slide it. Sewam then we came back as our Bye Bye We are censified from severalention. 


Saved transcription as /content/drive/MyDrive/WhisperAudio//TextFiles/New Recording 3.txt
Processing New Recording or Nezyyyzh.m4a...


/tmp/ipython-input-2017401303.py:27: UserWarning: PySoundFile failed. Trying audioread instead.
  audio, sr = librosa.load(file, sr=16000, mono=True)
/usr/local/lib/python3.12/dist-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)


1 words processed
Transcription of New Recording or Nezyyyzh.m4a:

 


Saved transcription as /content/drive/MyDrive/WhisperAudio//TextFiles/New Recording or Nezyyyzh.txt
Processing New Ltd Ybhfhflgyly Hubb.m4a...


...

1 words processed
8 words processed
10 words processed
13 words processed
16 words processed
17 words processed
18 words processed
20 words processed
28 words processed
30 words processed
31 words processed
33 words processed
37 words processed
38 words processed
39 words processed
40 words processed
49 words processed
50 words processed
51 words processed
60 words processed
66 words processed
68 words processed
70 words processed
74 words processed
75 words processed
80 words processed
83 words processed
87 words processed
88 words processed
89 words processed
90 words processed
91 words processed
Transcription of David-SmithphoneFebruary-17,-2021-at-12:39-PM8:00-.m4a:

My name is David Smith. I'm a at 100phone. and I have to where we. forward. on. No su- Goldenberg was going back to court on Monday One second. Bye-bye. interview you Please call me at... 212 9828 one. So I could interview you on the phone today. time. again My name is David Schmesel, I'm a Scabellion of can run out an

/tmp/ipython-input-2017401303.py:27: UserWarning: PySoundFile failed. Trying audioread instead.
  audio, sr = librosa.load(file, sr=16000, mono=True)
/usr/local/lib/python3.12/dist-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)


1 words processed
7 words processed
10 words processed
17 words processed
18 words processed
20 words processed
23 words processed
28 words processed
31 words processed
38 words processed
44 words processed
46 words processed
48 words processed
51 words processed
52 words processed
Transcription of Michael-SoshnickcellularFebruary-21,-2021-at-8:57-AM0:00-.m4a:

Good morning, Joseph. It's Michael Sashnick. in New York. Just reminding you to log in tomorrow. five. to 12. New York Times. if you have any questions. Please call me. I trust that you have the link. and that everything is in order. for tomorrow's. Virtual sciencing. Thank you, Joseph. care. Bye-bye. 


Saved transcription as /content/drive/MyDrive/WhisperAudio//TextFiles/Michael-SoshnickcellularFebruary-21,-2021-at-8:57-AM0:00-.txt
Processing +^(310)481-4099bpangne-20ha-at-a:28prOctober-29,-0:00-.m4a...


/tmp/ipython-input-2017401303.py:27: UserWarning: PySoundFile failed. Trying audioread instead.
  audio, sr = librosa.load(file, sr=16000, mono=True)
/usr/local/lib/python3.12/dist-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)


1 words processed
5 words processed
6 words processed
7 words processed
12 words processed
16 words processed
18 words processed
19 words processed
20 words processed
21 words processed
24 words processed
25 words processed
27 words processed
28 words processed
29 words processed
31 words processed
Transcription of +^(310)481-4099bpangne-20ha-at-a:28prOctober-29,-0:00-.m4a:

I just message for. Johnson? Goldenberg. And this is Grow Walker. You have to contact. Which one? 310-481-3103 for. Um onomy, or interactivity. employment. job searches. Okay. Please. Call me. Thank you. 


Saved transcription as /content/drive/MyDrive/WhisperAudio//TextFiles/+^(310)481-4099bpangne-20ha-at-a:28prOctober-29,-0:00-.txt
Moved /content/drive/MyDrive/WhisperAudio/New Recording 3.m4a to /content/drive/MyDrive/WhisperAudio/ProcessedAudio/
Moved /content/drive/MyDrive/WhisperAudio/New Recording or Nezyyyzh.m4a to /content/drive/MyDrive/WhisperAudio/ProcessedAudio/
Moved /content/drive/MyDrive/WhisperAudio/